In [1]:
!pip install catboost

In [1]:
import multiprocessing
multiprocessing.cpu_count()

12

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from catboost import CatBoostRanker,CatBoostRegressor,CatBoostClassifier,Pool,cv

In [ ]:
data = pd.read_parquet('all_train_gbm.parquet')

In [ ]:
data = data.drop(['user_id','comunity'],axis=1)

In [ ]:
data['query'].max()

In [ ]:
data

In [8]:
train_df = data[data['query'] <= 59999]
test_df = data[data['query'] > 59999]

In [9]:
train_df = train_df.fillna('NuN')#[['query','scor','label']]
test_df = test_df.fillna('NuN')#[['query','scor','label']]

In [10]:
import gc
gc.collect()

0

In [11]:
for col in ['bp_th_top1','bp_th_top2','bp_th_top3','themeid']:
    train_df[col] = train_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)

In [12]:
query = 'query'
label = 'label'
cat_cols = ['type','business_category','business_parent','type_ft','themeid',
           'bs_ct_top1','bs_ct_top2','bs_ct_top3','bp_ct_top1',
           'bp_ct_top2','bp_ct_top3','bp_th_top1',
           'bp_th_top2','bp_th_top3']

In [13]:
train_df['label'].value_counts()

0    2916262
1      83738
Name: label, dtype: int64

## Regression

In [16]:
train_pool = Pool(data=train_df.drop([query,label],axis=1),
                  label=train_df[label],
                  #cat_features=cat_cols,
                  group_id=train_df[query])

In [17]:
eval_pool = Pool(data=test_df.drop([query,label],axis=1),
                  label=test_df[label],
                  #cat_features=cat_cols,
                  group_id=test_df[query])

In [14]:
params = {'iterations':700,
          'learning_rate':0.01,
          'loss_function':'RMSE',
          'eval_metric':'MAE',
          'max_depth':5,
          'random_seed':56,
          'task_type':'GPU'}

model = CatBoostRegressor(**params)

In [16]:
model.fit(train_pool,eval_set=eval_pool,verbose=5)

CatBoostError: /src/catboost/catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'

In [ ]:
model.get_feature_importance(prettified=True)

## Ranker

In [16]:
train_pool = Pool(data=train_df.drop([label,query],axis=1),
                  label=train_df[label],
                  cat_features=cat_cols,
                  group_id=train_df[query])

In [17]:
eval_pool = Pool(data=test_df.drop([label,query],axis=1),
                  label=test_df[label],
                  cat_features=cat_cols,
                  group_id=test_df[query])

In [18]:
params = {'iterations':1600,
          'learning_rate':0.1,
          'loss_function':'PairLogitPairwise',
          'max_depth':6,
          'eval_metric':'PrecisionAt:top=7',
          'random_seed':56,
          'task_type':'GPU'}

model = CatBoostRanker(**params)

In [19]:
model.fit(train_pool,eval_set=eval_pool,verbose=25,plot=False)

Groupwise loss function. OneHotMaxSize set to 10


Default metric period is 5 because PrecisionAt is/are not implemented for GPU
Metric PrecisionAt:top=7 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.0845238	test: 0.0851143	best: 0.0851143 (0)	total: 209ms	remaining: 5m 33s
25:	learn: 0.0981452	test: 0.0981143	best: 0.0981143 (25)	total: 1.46s	remaining: 1m 28s
50:	learn: 0.0998548	test: 0.0994714	best: 0.0994714 (50)	total: 2.71s	remaining: 1m 22s
75:	learn: 0.1020214	test: 0.1016429	best: 0.1016429 (75)	total: 3.99s	remaining: 1m 19s
100:	learn: 0.1041786	test: 0.1038143	best: 0.1038143 (100)	total: 5.24s	remaining: 1m 17s
125:	learn: 0.1059976	test: 0.1059000	best: 0.1059000 (125)	total: 6.48s	remaining: 1m 15s
150:	learn: 0.1076143	test: 0.1071571	best: 0.1071571 (150)	total: 7.72s	remaining: 1m 14s
175:	learn: 0.1087881	test: 0.1083714	best: 0.1083714 (175)	total: 9s	remaining: 1m 12s
200:	learn: 0.1099238	test: 0.1094143	best: 0.1094143 (200)	total: 10.2s	remaining: 1m 11s
225:	learn: 0.1112286	test: 0.1104429	best: 0.1104429 (225)	total: 11.5s	remaining: 1m 10s
250:	learn: 0.1118738	test: 0.1110857	best: 0.1111000 (245)	total: 12.8s	remaining: 1m 8s
275:	learn: 0

In [20]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [21]:
predict_df = pd.read_parquet("all_pred_gbm.parquet")

In [22]:
predict_df

,query,user_id,scor,rank,comunity,customers_count,messages_count,type,business_category,business_parent,...,bp_ct_top3,bp_th_top1,bp_th_top2,bp_th_top3,std_customers,min_customers,max_customers,std_mess_count,min_mess_count,max_mess_count
0,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.760456,0,768680a00db384987ff7532abc1ebc8062281b4ac60124...,35518,2,7,FILM,PERSON,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
1,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.534067,1,390a9c1054f80353840567b156cb0b703d3c2a963a5d70...,21676,1,7,INTERNET_MEDIA,MEDIA,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
2,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.481027,2,6b1a156739da983b1d56d32cc8cb128b5b2a1c829de63c...,13858,14,7,TOURIST_AGENCY,TOURISM,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
3,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.440715,3,4ec66521f946fe87d1f3d75e5ef39e01fb6b89996fbe8d...,13206,9,7,LAW_OFFICE,PROFESSIONAL_SERVICES,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
4,0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,0.386041,4,53bbe4405c6abfe03092a641607e70637bfafdf420b088...,11226,7,7,ANIMALS,PETS,...,BLOG,NaN,NaN,62.0,18525.650359,182,73843,7.062444,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452295,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.076348,10,c03bf9aa00eb488975523a073ed4104a26f22e4d990886...,23280,2,7,BLOGGER,PERSON,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
3452296,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.073375,23,6651ca2f3d0b27d0fe06bab11caa4fad73ee6a91613c3a...,9699,3,7,PUBLIC_ORGANIZATION,SOCIAL_ORGANISATIONS,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
3452297,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.074058,21,2de900faf69694fdb245d99881502d8539b590f1126958...,10129,0,7,CARS_MACHINERY,AUTO,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4
3452298,5302,28e5354f7ffab3c1238dbf1b17bb2397d511b71ced4ea6...,0.071528,38,a93c51bf7a218a97d04167b943502fdaebcca683c9bb7c...,9401,0,7,CAMPING,LEISURE_AND_ENTERTAINMENT,...,SOCIAL_ORGANISATIONS,NaN,NaN,NaN,1083.512547,261,3043,1.200000,1,4


In [23]:
for col in ['bp_th_top1','bp_th_top2','bp_th_top3','themeid']:
    #train_df[col] = train_df[col].astype(str)
    predict_df[col] = predict_df[col].astype(str)

In [24]:
predict_df = predict_df.fillna('NuN')

In [27]:
predict_pool = Pool(data=predict_df.drop([query,'user_id','comunity'],axis=1),
                  cat_features=cat_cols,
                  group_id=predict_df[query])

In [28]:
predict_df['ranks'] = model.predict(predict_pool)

In [29]:
predict_df = predict_df.sort_values(by='query')

In [30]:
predict_df['query'].max() + 1

69046

In [31]:
model_ranks,als_ranks = [],[]
for qrt in tqdm(range(69046)):
    df_ = predict_df[qrt*50:(qrt+1)*50]
    als_ranks += [df_.sort_values(by='scor')['comunity'][::-1][:7].tolist()]
    model_ranks += [df_.sort_values(by='ranks')['comunity'][::-1][:7].tolist()]

  0%|          | 0/69046 [00:00<?, ?it/s]

/tmp/ipykernel_632/3886554370.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  model_ranks += [df_.sort_values(by='ranks')['comunity'][::-1][:7].tolist()]
/tmp/ipykernel_632/3886554370.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  als_ranks += [df_.sort_values(by='scor')['comunity'][::-1][:7].tolist()]


In [32]:
def get_submit(predict_users,preds):
    df = []
    for usr,prds in zip(predict_users,preds):
        fc = dict()
        fc['customer_id'] = usr
        for i in range(7):
            fc[f'community_id_{i+1}'] = prds[i]
        df += [fc]
    return pd.DataFrame(df)

In [33]:
submit = get_submit(predict_df['user_id'].unique(),model_ranks)

In [34]:
submit.to_csv('submit.csv',index=False)

In [35]:
submit

,customer_id,community_id_1,community_id_2,community_id_3,community_id_4,community_id_5,community_id_6,community_id_7
0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,768680a00db384987ff7532abc1ebc8062281b4ac60124...,dbdfeaeebe658d7a866ffa073158295685762b33629512...,005ca7b90a9fdea643eb6be501b289e6d451ca6d2f28dc...,95f663e12ea58f926d92f764b07674a53b29a5af1f5e2e...,4d5a53b97c13d64138565f8908be967a831646a708633c...,4ec66521f946fe87d1f3d75e5ef39e01fb6b89996fbe8d...,db65cd130a71bd957a707224d9414cd850f6022c2c5abd...
1,73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...,b0dadd3ca6aa79e83ede40fe5144e46b04efe1fd37fb67...,7e72a05405362daec036b6c918ec70645f99b09dd7ee2a...,68c88a68794384b8953398fbd06791a10c83acfb1e27d0...,1d4cceb7fafde4718d26a7a4a071a8b146154d95d5db59...,f146596214865eb9e4abf6a4d494480e4ad79a0191bd25...,e9fb3daa203599383c50c52421b7415230193116687111...,c857fdb0cd63d50e97e28c0e05d2dec31f985522bdd76b...
2,6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...,fb0c97fc02e05328718516409e97b19e72e9d544be1ed7...,092fd2a2cd45c5b51fcf94d610072cdf471e2d9522f5b1...,a8b15b543469eebe27c67c0c5897f6dc18e160cc5997c0...,892f2c17a6b9684c4dfd9834c0c6c8e7694c9faa00a0b3...,16183b4e07446d00c10d26deb9a1d62814d67fb6039b59...,bf66ba83213642a3060dfcde377d95c6b74f2bccc5a4a0...,0fcb8e67c30781ff9cf7f8290eff13144218d4d50ada43...
3,250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...,892f2c17a6b9684c4dfd9834c0c6c8e7694c9faa00a0b3...,a8b15b543469eebe27c67c0c5897f6dc18e160cc5997c0...,16183b4e07446d00c10d26deb9a1d62814d67fb6039b59...,abf30b1c3a454eaff00dea8e353bdef6c41ca9ea303ca6...,6729664ac03bd48062612647597252d30d097b79e8308b...,a8d0b95bb7330d3a1171c439c5db88e1a3671edfbca950...,4c9a186c14bbb29d71c24d48b56e7b8d05d5fc5e00c4f0...
4,2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...,b8ed0e918d9f11140c70ef08be19c197917f4f4455f255...,aee65cc087f56024df2aedd0755db2a3d193cf2ca558b7...,b4823c708c5fdba713f8f150327ce2101a5061d4f9d1f7...,ccc28c0a120ead6ec1cc77fb0555bf0195e28e3879b6ef...,17ecd45163fcf3ce895db4edf74a115e91779b26bb44ee...,ddbb0ee9f1eff9b9ff830faac9db11f178687e332eb28b...,14d2fb073c9a2929f1fd9919cbbae50a5c597d2459d7f5...
...,...,...,...,...,...,...,...,...
69041,606d6caf18a6209acaa46a1a0233a4bf22bc9885d70397...,e3adbab2d48db35c1a50f513ccc5416618fad8ef1e0567...,5d940e31b7c79eada0da34ff4dad5ab58e7694efca1b94...,13bc935e69c0119a69b6cb8f2a75e0150aa6c579fa826b...,57d99404c90c9344d3aec2c15ffb7b1a0f46c8a4f7157a...,586100a57969a348929446241ebaf5778c17d1546e3beb...,65645d4fb18a28c9a2726e736b3c881d4e3599be2b1f2b...,98d3a3519a33e4f56c6b8bf8eb3139a31eb691371fa45d...
69042,a71392c55f8bace9bd108a1acbf9c2335d700fb041966b...,dac62fcdc52f97daad2712837a808e304fd011bd5ec665...,58d35c987a0553d27bd68569818a8ad84e93db409c32c9...,b2e550048439263d779622fc9b4d221827573e9c6c5900...,b274af0e3a0afc5445b804d67cb2e789bbfff563c4478e...,7dab772c40d0df013a66aacf7f81f8cc446d7ff50cac2d...,3c8c56c125be2575c74750926654d11289941ff9b3f75e...,f7c22598d4d68bc4478d3b07c5f1dfee87e77bbcbf6169...
69043,7b6d03389208df699aa116563dc5c45cbf5c725b47b415...,13ac2f3d04f9306b4e3cc82e69b3f8e69f4dc7f12cf42d...,7e95f5c89f6c638a50bc6f6ebd54616a85c22db0e06e6e...,6682dfc418774213023298fe0618332b85e6f4f2095e62...,d5bf1f56574d7af705d4cd8b882ef2456cc857faa7bdfa...,01c1bd2b0bd1e7e1c8868a40add460f16d572859bdd6f5...,278cd81ddcb443f85149c70b320314842f8ba5a027c03f...,cbd6124cd10e3b6bdbe052d657002243bd45bd6d7c644b...
69044,8d1c564ba9f96dafb0a856fbc65fc4f9fc020863fe3b3f...,0664b5dc3168881dd7b4b1390ac80d618d7b0ce4afd8e5...,51b354f4db4e89508fad99ff9b466db96a307c9d1a1941...,bd7df33800e29e0d3c36814438d86a7e52886d80e43e76...,61199b60558bc8f9a89dcfcc7e5777edd38be1330bcda1...,72f77b9e0238d6951c32bd2917ab067d2c3f6ffb4746cd...,9f572e1f39b73a53ec0034d6ccf2ea7244b67a620457f3...,872892c975286011a5bdbf41ccbba8f1a1331a06097d23...


In [36]:
!zip submit.csv.zip submit.csv

updating: submit.csv (deflated 50%)


In [37]:
predict_df['user_id']

0          d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
27         d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
28         d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
29         d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
30         d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
                                 ...                        
1731975    f57918286b0047d2884e72631b65b4c7f13f5ffc697743...
1731974    f57918286b0047d2884e72631b65b4c7f13f5ffc697743...
1731973    f57918286b0047d2884e72631b65b4c7f13f5ffc697743...
1731971    f57918286b0047d2884e72631b65b4c7f13f5ffc697743...
1731998    f57918286b0047d2884e72631b65b4c7f13f5ffc697743...
Name: user_id, Length: 3452300, dtype: object